# LLM Component
Trying out the Hugging Face LLM model to use it as a Open-Ended Task Decorder.

In [16]:
from pycparser.ply.yacc import token
from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch

The Bart LLM is mainly used to generate text from given tokens or text. For the LLM task decoder of my Vision LLM it would be better to use a model that can generate a description or an answer based on a certain instruction. We can choose between the T5 or the Bert model. We will use the T5 model because it is specialized in giving descriptions and giving open-ended answers.

In [18]:
tokenizer = T5Tokenizer.from_pretrained('t5-base')

ImportError: 
T5Tokenizer requires the SentencePiece library but it was not found in your environment. Checkout the instructions on the
installation page of its repo: https://github.com/google/sentencepiece#installation and follow the ones
that match your environment. Please note that you may need to restart your runtime after installation.


In [3]:
bart_model = T5ForConditionalGeneration.from_pretrained('t5-base')

model.safetensors:   0%|          | 0.00/558M [00:00<?, ?B/s]

The tokenizer converts the textual format to tokens, a format the bart model can understand. 
The bart_model is the actual neural network that recieves the tokenized input and generates an output.

Lets try with a basic query:

In [5]:
text_input = "What is the most visited city in the world"
input_ids = tokenizer(text_input, return_tensors="pt").input_ids
input_ids

tensor([[   0, 2264,   16,    5,  144, 3790,  343,   11,    5,  232,    2]])

In [10]:
output_ids = bart_model.generate(input_ids, max_length=50, num_beams=4, early_stopping=True)
output_ids

tensor([[   2,    0, 2264,   16,    5,  144, 3790,  343,   11,    5,  232,    2]])

In [11]:
text_output = tokenizer.decode(output_ids[0], skip_special_tokens=True)
text_output

'What is the most visited city in the world'